In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://buomsoo-kim.github.io/colab/2020/05/09/Colab-mounting-google-drive.md/

NOte : Only use those lines of code if you import it in Google Colaboratory !

# GROUP PROJECT  - WEBSCRAPPING PHASE 1 - SCRAPPING DATA & CREATE A CSV FILE THAT CONTAINS THE DATA ABOUT REAL ESTATE

real estate link 

https://www.realtor.com/realestateandhomes-search/Portland_OR  

complete guide for web scraping 

https://www.webscrapingapi.com/real-estate-web-scraper

https://medium.com/rakesh-nain/using-python-collecting-real-world-data-by-web-scraping-real-estate-website-and-doing-data-2feb50a3b94f

getting an API proxi 

https://scrapeops.io/app/proxy
https://app.webscrapingapi.com/1/dashboardService



## 1 ) Preparing the work space : 
 
 importing the module we are going to use : beautifulsoup, pandas, requests

 https://www.webscrapingapi.com/real-estate-web-scraper#prepare-the-workspace

In [ ]:
# importing the BS4 and requests modules
from bs4 import BeautifulSoup 
import pandas as pd
import requests

## 2) Declaring the variables

 - here we have 6 variables : **1 integer** (page) take the page number to integrate in the url of the real estate website. Note that for obvious reasons we will instantiate the value of the variable page as 1 for the first page.

 - **5** variables(prices, beds, baths, sizes, and addresses) that we want to fetch from the real estate website. Note that those variables are of **list type** which is practicle to transfert on a csv file later on using the Pandas library. Those variables will be later on in the code populating by the values that we fetch on the website.

In [ ]:
page = 1
# column for the csv file and variable of type list, we will use pandas to transfer into the file
prices = []
beds = []
baths = []
sizes = []
addresses = []


## 3) Looping through the pages

1. we begin the loop by telling python to execute the loop as long as the variable page has not a value of 86. Why 86 ? because there are 86 pages on the real estate website. 
2. The **'html.parser**' tells python that the website content is written in HTML.
3. Once we have the content of the page in the variable soup, we create another variable labeled lists, we populate the variable by using the function find_all from Beautifulsoup. We give to to the function two parameters : the **class parameter** of the element we want to fetch('**li**' class here) which is the 'house card' , each house card contains all the element we wants (price, address, number of bed, number of bathrooms, size). and the second element we give to the function '**find_all**'. why do we use the find all ? because there is more that one house in a page. the second parameter is the name of the section. We past the html label from the source code of the real estate page.
4. once we have a house we start another loop (nested for loop) we ask python to use the find function to fetch all the elements that we need. we don't use the functon find all because for each house there is only one price, one set of bedrooms, bathrooms, size, one address. 
5. the function find used to parse each elements of the house take some parameters such as the class of the element and the label. we ask python to find one elemnt with a certain class who is labelled a certain way. 
6. At the end of the loop, we update the number of the page. For instance, after the first loop, the number of the page will havce a value of 2 which will change the url of the webpage. the second page of the real estate website will be send in a new request and cycle to fetch houses. 


multi page scraping 
https://www.geeksforgeeks.org/how-to-scrape-multiple-pages-of-a-website-using-python/



In [ ]:
while page!=86:
    # our API proxi that is sent with our request 
    url = "https://api.webscrapingapi.com/v1"
    params = {"api_key": "eBiY1EnI2X1RnBFG7mYXpoNhdmrLLhPN","url": f"https://www.realtor.com/realestateandhomes-search/Portland_OR/pg-{page}"}
    res = requests.request("GET", url, params=params)

    # beautiful soup fetch the content of the page in a variable soup
    soup = BeautifulSoup(res.content, 'html.parser')

    lists = soup.find_all('li', class_='jsx-1881802087 component_property-card')

    for element in lists:
      price = element.find('span', attrs={'data-label': 'pc-price'})
      bed = element.find('li', attrs={'data-label': 'pc-meta-beds'})
      bath = element.find('li', attrs={'data-label': 'pc-meta-baths'})
      size = element.find('li', attrs={'data-label': 'pc-meta-sqft'})
      address = element.find('div', attrs={'data-label': 'pc-address'})
      if bed and bath:
        nr_beds = bed.find('span', attrs={'data-label': 'meta-value'})
        nr_baths = bath.find('span', attrs={'data-label': 'meta-value'})
        if nr_beds and float(nr_beds.text) >= 2 and nr_baths and float(nr_baths.text) >= 1:
           beds.append(nr_beds.text)
           baths.append(nr_baths.text)

           if price and price.text:
               prices.append(price.text)
           else:
               prices.append('No display data')

           if size and size.text:
               sizes.append(size.text)
           else:
               sizes.append('No display data')

           if address and address.text:
               addresses.append(address.text)
           else:
               addresses.append('No display data')  
          
    page = page + 1


## 3) copy the data into a csv file using the pandas library.

The elemnents that we have fetched have populated the 5 variables of list type mentioned in the second step of that report. WE create the column name and we associate each column with the variable ad hoc. the second line of the code create a file name 'listing.csv' that we eill use later on

In [ ]:
df = pd.DataFrame({'Address': addresses, 'Price': prices, 'Beds': beds, 'Baths': baths, 'Sizes': sizes})
df.to_csv('listings.csv', index=False, encoding='utf-8')   

